In [1]:
from datasets import load_dataset, concatenate_datasets
import json
import torch
from datasets.utils.logging import disable_progress_bar,enable_progress_bar

dataset_name = "super_glue"  # The MMLU dataset name in Hugging Face Datasets
dataset = load_dataset(dataset_name, name='boolq')#["train"]
# print(dataset)

labels = dataset["train"].features["label"]
# def create_label_str(batch):
#     return {"label_str": labels.int2str(batch["label"])}
def create_label_str(example):
    return {"label_str": "Yes" if example["label"]== 1 else "No"}
# print(labels)


/home/hshapour/.pyenv/versions/3.10.12/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 3245/3245 [00:00<00:00, 21592.55 examples/s]


In [16]:
from datasets import Dataset
my_list = [{"a": 1}, {"a": 2}, {"a": 3}]
dataset = Dataset.from_list(my_list)
print(dataset)

Dataset({
    features: ['a'],
    num_rows: 3
})


In [2]:

# data_train = dataset["train"].map(create_label_str)   
# data_test = dataset["validation"].map(create_label_str)   
# # print(d1)
# # dd = d1.select(range(5)).shuffle()

d1 = dataset["train"].map(create_label_str)   
d2 = dataset["validation"].map(create_label_str)   

# d_all = concatenate_datasets([d1,d2])
data_train = d1.select(range(5))
data_test = d2 #.select(range(5,d2.num_rows))

print(data_train.num_rows, data_test.num_rows)

Map: 100%|██████████| 3270/3270 [00:00<00:00, 13684.01 examples/s]

5 3270


In [3]:


def json_writer(JSON_PATH, num_qs, n_shot_example,seed=10,space=False):
    qlist = {}
    # if n_shot_example >0:
    #     instruction = 'The following are multiple choice questions (with answers) about  '
    # else:
    #     instruction = 'The following is a multiple choice question about  '
    with open(JSON_PATH, 'w') as json_file:

        for ind in range(num_qs):
            dd_examples = data_train.select(range(5)).shuffle()
            prompt = ""#"Answer the question from the passage with Yes or No.\n\n"
            
            for idx in range(n_shot_example):
                prompt += "Answer the question from the passage with Yes or No.\n\n"
                prompt += "Passage: " + dd_examples[idx]['passage'] + '\n'
                prompt += "Question: " + dd_examples[idx]['question'] + '\n'
                prompt += "Answer:" + dd_examples[idx]['label_str'] + '\n\n'

            prompt += "Answer the question from the passage with Yes or No.\n\n"
            prompt += "Passage: " + data_test[ind]['passage'] + '\n'
            prompt += "Question: " + data_test[ind]['question'] + '\n'
            prompt += "Answer:" 
    
            
            qlist['input'] = prompt
            qlist['target_str'] = data_test[ind]['label_str']
            qlist['target'] = data_test[ind]['label']

            # Serializing json
            json_file.write(json.dumps(qlist) + '\n')
        # print("Done!")

In [4]:
disable_progress_bar()

# print(example_idx)
n_example = data_test.num_rows
n_shot_vals = [2]
seed = 123
for n_shot in range(6):
# for n_shot in n_shot_vals:
    print(f"{n_shot}-shot example is being generated...")
    fname_json = f"boolq-data/{n_shot}_shot_examples.json"
    # fname_json = f"boolq-data/{n_shot}_shot_examples_small.json"
    # JSON_eval = 'sample_eval.json'
    json_writer(fname_json, n_example, n_shot, seed=seed, space= False)
    # json_writer(JSON_eval,example_idx[(n_shot_example+1)*n_test_ex:])

enable_progress_bar()


0-shot example is being generated...
1-shot example is being generated...
2-shot example is being generated...
3-shot example is being generated...
4-shot example is being generated...
5-shot example is being generated...


In [6]:
# fname_json = f"mmlu-data/ex_no_space.json"
n_shot = 1
d1 = load_dataset("json", data_files={
        'dev' : f"boolq-data/{n_shot}_shot_examples.json"
    })
# print(d1['dev'])
# print(d1['dev'].filter(lambda example: example["subject"] == subject))
# d2 = d1['dev'].filter(lambda example: example["subject"] == subject)
# # print(d2['input'][0][-1])
# # print("---------")
print(d1['dev'][2]['input'])

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 706.11it/s]
Generating dev split: 3270 examples [00:00, 121132.73 examples/s]

Answer the question from the passage with Yes or No.

Passage: The Elder Scrolls Online -- As with other games in The Elder Scrolls series, the game is set on the continent of Tamriel. The events of the game occur a millennium before those of The Elder Scrolls V: Skyrim and around 800 years before The Elder Scrolls III: Morrowind and The Elder Scrolls IV: Oblivion. It has a broadly similar structure to Skyrim, with two separate conflicts progressing at the same time, one with the fate of the world in the balance, and one where the prize is supreme power on Tamriel. In The Elder Scrolls Online, the first struggle is against the Daedric Prince Molag Bal, who is attempting to meld the plane of Mundus with his realm of Coldharbour, and the second is to capture the vacant imperial throne, contested by three alliances of the mortal races. The player character has been sacrificed to Molag Bal, and Molag Bal has stolen their soul, the recovery of which is the primary game objective.
Question: 

In [7]:
from datasets import DatasetDict, Dataset
from collections import defaultdict

fname_dict = {}
for n_shot in range(6):
    fname_dict[f"{n_shot}_shot"] = f"boolq-data/{n_shot}_shot_examples.json"
# data_all = defaultdict(DatasetDict)
d_upload = load_dataset("json", data_files=fname_dict)
    # data_all[f"n_shot = {n_shot}"] = d1['dev']


Extracting data files: 100%|██████████| 6/6 [00:00<00:00, 1777.00it/s]
Generating 0_shot split: 3270 examples [00:00, 288671.79 examples/s]
Generating 1_shot split: 3270 examples [00:00, 194400.93 examples/s]
Generating 2_shot split: 3270 examples [00:00, 121071.78 examples/s]
Generating 3_shot split: 3270 examples [00:00, 99454.51 examples/s]
Generating 4_shot split: 3270 examples [00:00, 75384.88 examples/s]
Generating 5_shot split: 3270 examples [00:00, 63178.58 examples/s]


In [8]:
# print(Dataset.from_dict(data_all))
# Dataset.from_dict(data_all).push_to_hub("hassansh/boolq_n_shot")
d_upload.push_to_hub("hassansh/boolq_n_shot")


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]
README.md: 100%|██████████| 1.34k/1.34k [00:00<00:00, 8.03MB/s]


In [38]:
print(d_upload['5_shot'][2]['input'])

Passage: Good Samaritan law -- Good Samaritan laws offer legal protection to people who give reasonable assistance to those who are, or who they believe to be, injured, ill, in peril, or otherwise incapacitated. The protection is intended to reduce bystanders' hesitation to assist, for fear of being sued or prosecuted for unintentional injury or wrongful death. An example of such a law in common-law areas of Canada: a good Samaritan doctrine is a legal principle that prevents a rescuer who has voluntarily helped a victim in distress from being successfully sued for wrongdoing. Its purpose is to keep people from being reluctant to help a stranger in need for fear of legal repercussions should they make some mistake in treatment. By contrast, a duty to rescue law requires people to offer assistance and holds those who fail to do so liable.
Question: do good samaritan laws protect those who help at an accident
Answer:Yes

Passage: Powdered sugar -- Powdered sugar, also called confectioner

In [24]:
num_shots = 1
dataset_name = "hassansh/multirc_n_shot"
dataset = load_dataset(dataset_name, name = f"{num_shots}_shot")['test']#.select(range(10))
print(dataset[10]['input'])

Based on the paragraph, determine whether the candidate answer to the question is True or False.

Paragraph: A stranger in town meets pretty young Susan Martinez De La Cruz and accompanies her to a barbecue , where wealthy Jason Carberry is saying a few words for the recently departed Robin Randall , a citizen who got shot .  Jason objects to the stranger's presence , being Susan's guardian and protective of her .  He challenges him to a shootout , but the stranger pulls his pistol before Jason's can even clear the holster .  Calaveras Kate , a saloon singer who's in love with Jason , is relieved when the stranger declines to pull the trigger .  Rafael Moreno suddenly rides into town and picks a fight with the stranger .  Their brawl continues until the arrival of Judge Wallace Wintrop and his niece , Sheila , who have come to town from back East and deplore all this random violence out West .  The stranger is recognized as Reb Randall , the dead man's brother .  He is looking for the 